<h1><center>
Toxic Comments Kaggle Competition Classifying
</center></h1>
<h2><center>
Modeling the Data
</center></h2>
<h3><center>
Marcel Colvin 912033961
</center></h3>

In [30]:
import pandas as pd
import numpy as np
import sklearn as sk
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
import time
import warnings
warnings.filterwarnings("ignore")

In [31]:
train = pd.read_csv("training_data.csv")
test = pd.read_csv("test_data.csv")

# MLP Classifier with w2v algorithm
### This had 95.04% accuracy

In [3]:
train["avg_word"] = train["avg_word"].apply(lambda x: 
                           np.fromstring(
                               x.replace('\n','')
                                .replace('[','')
                                .replace(']','')
                                .replace('  ',' '), sep=' '))

In [4]:
test["avg_word"] = test["avg_word"].apply(lambda x: 
                           np.fromstring(
                               x.replace('\n','')
                                .replace('[','')
                                .replace(']','')
                                .replace('  ',' '), sep=' '))

In [5]:
train_matrix = np.zeros((len(train),300), dtype="float64")
test_matrix = np.zeros((len(test),300), dtype="float64")
for i in range(len(train)):
    train_matrix[i] = train.iloc[i]["avg_word"].reshape((1,300))
for i in range(len(test)):
        test_matrix[i] = test.iloc[i]["avg_word"].reshape((1,300))

In [6]:
y_train = [train["toxic"], train["severe_toxic"], train["obscene"], train["threat"], train["insult"], train["identity_hate"]]

In [7]:
x_train = train["avg_word"].values

In [9]:
model = []
for i in range(6):
    m = MLPClassifier(hidden_layer_sizes=(30,30,30), random_state=5)
    model.append(m)

In [10]:
for i in range(6):
    model[i].partial_fit(train_matrix, y_train[i], [0,1])

/home/marcel/anaconda3/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:927: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if np.setdiff1d(classes, self.classes_, assume_unique=True):
/home/marcel/anaconda3/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:927: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if np.setdiff1d(classes, self.classes_, assume_unique=True):
/home/marcel/anaconda3/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:927: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
 

In [11]:
result = test[['id']].copy() 

In [12]:
result['toxic'] = model[0].predict_proba(test_matrix)[:,1]
result['severe_toxic'] = model[1].predict_proba(test_matrix)[:,1]
result['obscene'] = model[2].predict_proba(test_matrix)[:,1]
result['threat'] = model[3].predict_proba(test_matrix)[:,1]
result['insult'] = model[4].predict_proba(test_matrix)[:,1]
result['identity_hate'] = model[5].predict_proba(test_matrix)[:,1]

In [ ]:
result.to_csv("submission.csv", index = False)

# TFIDF Logistitc Regression, "comment_text" uncleaned
### This had 93.94% accuracy

In [16]:
tfidfvect = TfidfVectorizer(max_features=5000,stop_words='english')

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=5000, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,
        stop_words='english', strip_accents=None, sublinear_tf=False,
        token_pattern='(?u)\\b\\w\\w+\\b', tokenizer=None, use_idf=True,
        vocabulary=None)

In [19]:
X = tfidfvect.fit_transform(train["comment_text"])

In [20]:
testX = tfidfvect.transform(test["comment_text"])

In [22]:
for i in range(6):
    m = LogisticRegression(C=10.0)
    model.append(m)

In [24]:
for i in range(6):
    model[i].fit(X,y_train[i])

In [25]:
result = test[['id']].copy() 

In [26]:
result['toxic'] = model[0].predict_proba(testX)[:,1]
result['severe_toxic'] = model[1].predict_proba(testX)[:,1]
result['obscene'] = model[2].predict_proba(testX)[:,1]
result['threat'] = model[3].predict_proba(testX)[:,1]
result['insult'] = model[4].predict_proba(testX)[:,1]
result['identity_hate'] = model[5].predict_proba(testX)[:,1]

In [29]:
result.to_csv("submission.csv", index = False)

# Cleaning Comment Text to run Logistic Regression Again